In [ ]:
# Set up packages for lecture. Don't worry about understanding this code,
# but make sure to run it if you're following along.
import numpy as np
import babypandas as bpd
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 5)

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option("display.max_rows", 7)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

from IPython.display import display, IFrame

def binning_animation():
    src="https://docs.google.com/presentation/d/e/2PACX-1vTnRGwEnKP2V-Z82DlxW1b1nMb2F0zWyrXIzFSpQx_8Wd3MFaf56y2_u3JrLwZ5SjWmfapL5BJLfsDG/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width=900
    height=270
    display(IFrame(src, width, height))

# Lecture 7 – Distributions and Histograms

## DSC 10, Fall 2023

### Announcements

- Quiz 1 scores are released!
    - Remember, it's only worth 5% of your grade, and will be dropped entirely if you do better on the remaining quizzes.
- Homework 1 is due **tomorrow at 11:59PM**.
- Lab 2 is coming out today and is due on **Thursday at 11:59PM**.
- Come to office hours for help! The schedule is [here](https://dsc10.com/calendar).
    - There are now Saturday office hours held in the CSE building (all other office hours are still in the HDSI building).

### Agenda

- Distributions.
- Density histograms.
- Overlaid plots.

Today's material is quite theoretical – make sure to go to discussion next week to practice with it!

### Review: Types of visualizations

The type of visualization we create depends on the kinds of variables we're visualizing.

- **Scatter plot**: Numerical vs. numerical.
    - Example: Weight vs. height.
- **Line plot**: Sequential numerical (time) vs. numerical.
    - Example: Height vs. time.
- **Bar chart**: Categorical vs. numerical.
    - Example: Heights of different family members.
- **Histogram**: Distribution of numerical.
    
We may interchange the words "plot", "chart", and "graph"; they all mean the same thing.

### Some bad visualizations

<br>

<center><img src="images/bananas.png" width=60%></center>

<center><img src="images/average_height.png" width=50%></center>

<center><img src="images/weather.png" width=80%></center>

## Distributions

### What is the distribution of a variable?

- The distribution of a variable consists of all values of the variable that occur in the data, along with their frequencies.
- Distributions help you understand:
    > _How often does a variable take on a certain value?_
- Both categorical and numerical variables have distributions.

### Categorical variables

The distribution of a categorical variable can be displayed as a table or bar chart, among other ways!

For example, let's look at the distribution of exoplanet `'Type'`s. To do so, we'll need to group.

In [ ]:
exo = bpd.read_csv('data/exoplanets.csv').set_index('Name')
exo

In [ ]:
# Remember, when we group and use .count(), the column names aren't meaningful.
type_counts = exo.groupby('Type').count()
type_counts

In [ ]:
# As a result, we could have set y='Magnitude', for example, and gotten the same plot.
type_counts.plot(kind='barh', y='Distance', 
                 legend=False, xlabel='Count', title='Distribution of Exoplanet Types');

Notice the optional `title` argument. Some other useful optional arguments are `legend`, `figsize`, `xlabel`, and `ylabel`. There are [many optional arguments](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html).

It looks like terrestrial exoplanets are the most rare in the dataset. In the last lecture, we also saw that they have the smallest average radius of any `'Type'`, too.

In [ ]:
exo.groupby('Type').mean().get('Radius')

Let's look into them further!

### Terrestrial exoplanets 🌑

In [ ]:
terr = exo[exo.get('Type') == 'Terrestrial']
terr

Let's focus on the `'Radius'` column of `terr`. To learn more about it, we can use the `.describe()` method.

In [ ]:
terr.get('Radius').describe()

But how do we visualize its distribution?

### Visualizing the distribution of `'Radius'`, a numerical variable

- A few slides ago, we looked at the distribution of `'Type'`, which is a categorical variable.
- Now, we'll look at the distribution of `'Radius'`, which is a numerical variable. 
- As we'll see, **a bar chart is not the right choice of visualization for the distribution of a numerical variable**.

To try and see the distribution of `'Radius'`, we need to group by that column and count how many terrestrial planets there are of each radius.

In [ ]:
terr_radius = terr.groupby('Radius').count()
terr_radius = (terr_radius
                 .assign(Count=terr_radius.get('Distance'))
                 .get(['Count'])
              )
terr_radius

In [ ]:
terr_radius.plot(kind='bar', y='Count', figsize=(15, 5));

The horizontal axis should be numerical (like a number line), not categorical. There should be more space between certain bars than others.

For instance, the planet with `'Radius'` 1.8 is 80% larger than the planet with `'Radius'` 1, but they appear to be about the same size here.

## Density histograms

### Density histograms show the distribution of numerical variables

Instead of a bar chart, we'll visualize the distribution of a numerical variable with a **density histogram**. Let's see what a density histogram for `'Radius'` looks like. What do you notice about this visualization?

In [ ]:
# Ignore the code for right now.
terr.plot(kind='hist', y='Radius', density=True, bins = np.arange(0, 3.5, 0.25), ec='w');

In [ ]:
# There are 7 terrestrial exoplanets with a radius of exactly 1.0,
# but the height of the bar starting at 1.0 is not 7!
terr[terr.get('Radius') == 1]

### First key idea behind histograms: Binning 🗑️

- Binning is the act of counting the number of numerical values that fall within ranges defined by two endpoints. These ranges are called “bins”.
- A value falls in a bin if it is **greater than or equal to the left** endpoint and **less than the right** endpoint.
    - [a, b): a is included, b is not.
- The width of a bin is its right endpoint minus its left endpoint.


In [ ]:
binning_animation()

### Plotting a density histogram

- **Density histograms** (not bar charts!) visualize the distribution of a single numerical variable by placing numbers into bins.
- To create one from a DataFrame `df`, use
```py
df.plot(
    kind='hist', 
    y=column_name,
    density=True
)
```
- Optional but recommended: Use `ec='w'` to see where bins start and end more clearly.

### Customizing the bins

- By default, Python will bin your data into 10 equally sized bins.
- You can specify another number of equally sized bins by setting the optional argument `bins` equal to some other integer value.
- You can also specify custom bin start and endpoints by setting `bins` equal to a list or array of bin endpoints.

In [ ]:
# There are 10 bins by default, some of which are empty.
terr.plot(kind='hist', y='Radius', density=True, ec='w');

In [ ]:
terr.plot(kind='hist', y='Radius', density=True, bins=20, ec='w');

In [ ]:
terr.plot(kind='hist', y='Radius', density=True, bins=[0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5], ec='w');

In the three histograms above, what is different and what is the same?

### Observations
- The general shape of all three histograms is the same, regardless of the bins.
- More bins gives a finer, more granular picture of the distribution of the variable `'Radius'`.
- The $y$-axis values seem to change a lot when we change the bins. Hang onto that thought; we'll see why shortly.

### Bin details

- In a histogram, only the last bin is inclusive of the right endpoint!
- The bins you specify don't have to include all data values; data values not in any bin won't be shown in the histogram.
- For equally sized bins, use `np.arange`.
    - Be **very careful** with the endpoints. 
    - For example, `bins=np.arange(4)` creates the bins [0, 1), [1, 2), [2, 3].
- Bins can have different sizes!

In [ ]:
terr.plot(kind='hist', y='Radius', density=True,
            bins=np.arange(0, 3.5, 0.5),
            ec='w');

In [ ]:
terr.sort_values('Radius', ascending=False)

In the above example, the terrestrial exoplanet with the largest radius (Kepler-33 c) is not included because the rightmost bin is [2.5, 3.0] and Kepler-33 c has a `'Radius'` of 3.13.

In [ ]:
terr.plot(kind='hist', y='Radius', density=True,
          bins=[0, 0.25, 0.5, 0.75, 2, 4]);

In the above example, the bins have different widths!

### Second key idea behind histograms: Total area is 1

- In a density histogram, the $y$-axis can be hard to interpret, but it's designed to give the histogram a very nice property: 
<br><br>
<center>
    <b>The bars of a density histogram<br>have a combined total area of 1.</b>
</center>

- Important: <b>The area of a bar is equal to the proportion of all data points that fall into that bin</b>.

- Recall from the pretest, proportions and percentages represent the same thing.
    - A proportion is a decimal between 0 and 1, a percentage is between 0\% and 100\%.
    - The proportion 0.34 means 34\%.

### Example calculation

In [ ]:
terr.plot(kind='hist', y='Radius', density=True,
          bins=[0, 0.25, 0.5, 0.75, 2, 4]);

Based on this histogram, what proportion of terrestrial exoplanets have a `'Radius'` between 0.5 and 0.75?

### Example calculation

- The height of the [0.5, 0.75) bar looks to be around 0.8.
- The width of the bin is 0.75 - 0.5 = 0.25.

- Therefore, using the formula for the area of a rectangle,

$$\begin{align}\text{Area} &= \text{Height} \times \text{Width} \\ &= 0.8 \times 0.25 \\ &= 0.2
\end{align}$$

- Since areas represent proportions, this means that the proportion of terrestrial exoplanets with a radius between 0.5 and 0.75 is about 0.2 (or 20\%).

### Check the math 🧮

In [ ]:
in_range = terr[(terr.get('Radius') >= 0.5) & (terr.get('Radius') < 0.75)].shape[0]
in_range

In [ ]:
in_range / terr.shape[0]

This matches the result we got. (Not exactly, since we made an estimate for the height.)

### Calculating heights in a density histogram

Since a bar of a histogram is a rectangle, its area is given by

$$\text{Area} = \text{Height} \times \text{Width}$$

That means

$$\text{Height} = \frac{\text{Area}}{\text{Width}} = \frac{\text{Proportion (or Percentage)}}{\text{Width}}$$

This implies that the units for height are "proportion per ($x$-axis unit)". The $y$-axis represents a sort of *density*, which is why we call it a density histogram.



In [ ]:
terr.plot(kind='hist', y='Radius', density=True,
          bins=[0, 0.25, 0.5, 0.75, 2, 4]);

The $y$-axis units here are "proportion per radius", since the $x$-axis represents radius.
  - Unfortunately, the $y$-axis units on the histogram always displays as "Frequency". **This is wrong!**
  - We can fix this with the optional argument `ylabel` but we usually don't.

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

Suppose we created a density histogram of people's shoe sizes. 👟 Below are the bins we chose along with their heights.

| Bin | Height of Bar |
| --- | --- |
| [3, 7) | 0.05 |
| [7, 10) | 0.1 |
| [10, 12) | 0.15 |
| [12, 16] | $X$ |


What should the value of $X$ be so that this is a valid histogram?

A. 0.02 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
B. 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
C. 0.2 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
D. 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
E. 0.7 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;


### Bar charts vs. histograms

Bar chart | Histogram
---|---
Shows the distribution of a categorical variable | Shows the distribution of a numerical variable
Plotted from 2 columns of a DataFrame | Plotted from 1 column of a DataFrame
1 categorical axis,  1 numerical axis | 2 numerical axes
Bars have arbitrary, but equal, widths and spacing | Horizontal axis is numerical and to scale
Lengths of bars are proportional to the numerical quantity of interest | Height measures density; areas are proportional to the proportion (percent) of individuals

### 🌟 Important 🌟

In this class, **"histogram" will always mean a "density histogram".** We will **only** use density histograms. 

*Note:* It's possible to create what's called a *frequency histogram* where the $y$-axis simply represents a count of the number of values in each bin. 

While easier to interpret, frequency histograms don't have the important property that the total area is 1, so they can't be connected to probability in the same way that density histograms can. This property will be useful to us later on in the course.

## Overlaid plots

### Multiple plots on the same axes

Let's look back at the `exo` DataFrame.

In [ ]:
exo

Can we look at both the average `'Magnitude'` and the average `'Radius'` for each `'Type'` at the same time?

In [ ]:
types = exo.groupby('Type').mean()
types

In [ ]:
types.get(['Magnitude', 'Radius']).plot(kind='barh');

How did we do that?

### Overlaying plots

When calling `.plot`, if we omit the `y=column_name` argument, **all other columns** are plotted.

In [ ]:
types

In [ ]:
types.plot(kind='barh');

### Selecting multiple columns at once
- To select multiple columns, use `.get([column_1, ..., column_k])`.
- Passing a list of column labels to `.get` returns a DataFrame.
    - `.get([column_name])` will return a DataFrame with just one column!

In [ ]:
types

In [ ]:
types.get(['Magnitude', 'Radius'])

In [ ]:
types.get(['Magnitude', 'Radius']).plot(kind='barh');

### Plotting multiple graphs at once

Recipe:
1. `.get` only the columns that contain information relevant to your plot (or, equivalently, `.drop` all extraneous columns).
1. Specify the column for the $x$-axis (if not the index) in `.plot(x=column_name)`.
1. Omit the `y` argument. Then **all** other columns will be plotted on a shared $y$-axis.

The same thing works for `'barh'`, `'bar'`, and `'hist'`, but not `'scatter'`.

### Another example: Heights of children and their parents 👪 📏

- The data below was collected in the late 1800s by [Francis Galton](https://en.wikipedia.org/wiki/Francis_Galton).
    - He was a eugenicist and proponent of scientific racism, which is why he collected this data.
    - Today, we understand that eugenics is immoral, and that there is no scientific evidence or any other justification for racism.
- We will revisit this dataset later on in the course.
- For now, we'll only need the `'mother'`, and `'childHeight'` columns.

In [ ]:
mother_child = bpd.read_csv('data/galton.csv').get(['mother', 'child'])
mother_child

### Plotting overlaid histograms

`alpha` controls how transparent the bars are (`alpha=1` is opaque, `alpha=0` is transparent).

In [ ]:
height_bins = np.arange(55, 80, 2.5)
mother_child.plot(kind='hist', density=True, ec='w',
                  alpha=0.65, bins=height_bins);

Why do children seem so much taller than their mothers?

### Extra Practice

Try to answer these questions based on the overlaid histogram.

1. What proportion of children were between 70 and 75 inches tall?

2. What proportion of mothers were between 60 and 63 inches tall?

<details>
    <summary>✅ Click here to see the answers to the problems above <b>after</b> you've tried them on your own.</summary>
    
<b>Question 1</b>
    
The height of the $[70, 72.5)$ bar is around $0.08$, meaning that $0.08 \cdot 2.5 = 0.2$ of children had heights in that interval. The height of the $[72.5, 75)$ bar is around $0.02$, meaning $0.02 \cdot 2.5 = 0.05$ of children had heights in that interval. Thus, the overall proportion of children who were between $70$ and $75$ inches tall was around $0.20 + 0.05 = 0.25$, or $25\%$.
    
To verify our answer, we can run

<code>mother_child[(mother_child.get('child') >= 70) & (mother_child.get('child') < 75)].shape[0] / mother_child.shape[0]</code>
    
<b>Question 2</b>
    
We can't tell. We could try and breaking it up into the proportion of mothers in $[60, 62.5)$ and $[62.5, 63)$, but we don't know the latter. In the absence of any additional information, we can't infer about the distribution of values within a bin. For example, it could be that everyone in the interval $[62.5, 65)$ actually falls in the interval  $[62.5, 63)$ - or it could be that no one does!

</details>

## Summary, next time

### Summary

- Histograms (not bar charts!) are used to display the distribution of a numerical variable.
- We will always use density histograms in this course.
    - In a density histogram, the area of a bar represents the proportion (percentage) of values within its bin.
    - The total area of all bars is 1 (100%).
- We can overlay multiple line plots, bar charts, and histograms on top of one another to look at multiple relationships or distributions.

### Next time

- Writing our own functions.
- Applying functions to the data in a DataFrame.